# bgee Human Sample

Author: Moshe Silverstein  
Date: 07-18  
Data Source Home: https://bgee.org/     
Data Source Download: https://bgee.org/?page=download&action=expr_calls#id1 

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import utility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

# Path to Output Files

In [3]:
path = '/Users/moshesilverstein/Documents/Harmonizome/bgee/Output/'

# Load Data

In [4]:
df = pd.read_csv('Input/Homo_sapiens_expr_advanced_development.tsv', sep='\t')

/Users/moshesilverstein/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,Gene ID,Gene name,Anatomical entity ID,Anatomical entity name,Developmental stage ID,Developmental stage name,Expression,Call quality,Expression rank,Including observed data,...,In situ hybridization experiment count showing expression of this gene in this condition or in sub-conditions with a low quality,In situ hybridization experiment count showing absence of expression of this gene in this condition or valid parent conditions with a high quality,In situ hybridization experiment count showing absence of expression of this gene in this condition or valid parent conditions with a low quality,Including in situ hybridization observed data,RNA-Seq data,RNA-Seq experiment count showing expression of this gene in this condition or in sub-conditions with a high quality,RNA-Seq experiment count showing expression of this gene in this condition or in sub-conditions with a low quality,RNA-Seq experiment count showing absence of expression of this gene in this condition or valid parent conditions with a high quality,RNA-Seq experiment count showing absence of expression of this gene in this condition or valid parent conditions with a low quality,Including RNA-Seq observed data
0,ENSG00000000003,TSPAN6,CL:0000015,male germ cell,HsapDv:0000092,human middle aged stage (human),present,silver quality,5940,yes,...,0,0,0,no,no data,0,0,0,0,no
1,ENSG00000000003,TSPAN6,CL:0000019,sperm,HsapDv:0000088,human early adulthood stage (human),present,silver quality,23,yes,...,0,0,0,no,no data,0,0,0,0,no
2,ENSG00000000003,TSPAN6,CL:0000023,oocyte,HsapDv:0000087,human adult stage (human),absent,silver quality,20500,yes,...,0,0,0,no,no data,0,0,0,0,no
3,ENSG00000000003,TSPAN6,CL:0000083,epithelial cell of pancreas,UBERON:0000104,life cycle,present,silver quality,6050,yes,...,0,0,0,no,no data,0,0,0,0,no
4,ENSG00000000003,TSPAN6,CL:0000115,endothelial cell,HsapDv:0000092,human middle aged stage (human),present,silver quality,11800,yes,...,0,0,0,no,no data,0,0,0,0,no


In [6]:
df.shape

(32962486, 32)

# Get relevant data

In [7]:
df = df[['Gene name', 'Anatomical entity name', 'Developmental stage name', 'Expression', 'Call quality']]

In [8]:
df.head()

,Gene name,Anatomical entity name,Developmental stage name,Expression,Call quality
0,TSPAN6,male germ cell,human middle aged stage (human),present,silver quality
1,TSPAN6,sperm,human early adulthood stage (human),present,silver quality
2,TSPAN6,oocyte,human adult stage (human),absent,silver quality
3,TSPAN6,epithelial cell of pancreas,life cycle,present,silver quality
4,TSPAN6,endothelial cell,human middle aged stage (human),present,silver quality


In [9]:
df = df[df['Expression'] == 'present']

In [10]:
df['Sample'] = df['Anatomical entity name'].map(str) +'_'+ df['Developmental stage name'].map(str)

In [11]:
df.head()

,Gene name,Anatomical entity name,Developmental stage name,Expression,Call quality,Sample
0,TSPAN6,male germ cell,human middle aged stage (human),present,silver quality,male germ cell_human middle aged stage (human)
1,TSPAN6,sperm,human early adulthood stage (human),present,silver quality,sperm_human early adulthood stage (human)
3,TSPAN6,epithelial cell of pancreas,life cycle,present,silver quality,epithelial cell of pancreas_life cycle
4,TSPAN6,endothelial cell,human middle aged stage (human),present,silver quality,endothelial cell_human middle aged stage (human)
7,TSPAN6,leukocyte,human middle aged stage (human),present,silver quality,leukocyte_human middle aged stage (human)


In [12]:
meta_data = df[['Sample', 'Anatomical entity name', 'Developmental stage name']].copy()

In [13]:
meta_data.set_index('Sample', inplace=True)

In [14]:
meta_data.drop_duplicates(inplace=True)

In [15]:
meta_data.head()

,Anatomical entity name,Developmental stage name
Sample,,
male germ cell_human middle aged stage (human),male germ cell,human middle aged stage (human)
sperm_human early adulthood stage (human),sperm,human early adulthood stage (human)
epithelial cell of pancreas_life cycle,epithelial cell of pancreas,life cycle
endothelial cell_human middle aged stage (human),endothelial cell,human middle aged stage (human)
leukocyte_human middle aged stage (human),leukocyte,human middle aged stage (human)


In [16]:
df = df[['Gene name', 'Sample']]

In [17]:
df.head()

,Gene name,Sample
0,TSPAN6,male germ cell_human middle aged stage (human)
1,TSPAN6,sperm_human early adulthood stage (human)
3,TSPAN6,epithelial cell of pancreas_life cycle
4,TSPAN6,endothelial cell_human middle aged stage (human)
7,TSPAN6,leukocyte_human middle aged stage (human)


In [18]:
df.shape

(19631082, 2)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [19]:
df.set_index('Gene name', inplace=True)

In [20]:
uf.mapgenesymbols(df)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Drop Duplicates

In [21]:
df.reset_index(inplace=True)

In [22]:
df.drop_duplicates(inplace=True)

In [23]:
df.head()

,Gene name,Sample
0,TSPAN6,male germ cell_human middle aged stage (human)
1,TSPAN6,sperm_human early adulthood stage (human)
2,TSPAN6,epithelial cell of pancreas_life cycle
3,TSPAN6,endothelial cell_human middle aged stage (human)
4,TSPAN6,leukocyte_human middle aged stage (human)


In [24]:
df.shape

(14983142, 2)

# Create Binary Matrix

In [25]:
binary_matrix = uf.createBinaryMatrix(df)

In [26]:
binary_matrix.head()

,epithelium of bronchus_65-79 year-old human stage (human),small intestine Peyer's patch_human late adulthood stage (human),frontal cortex_6-12 year-old child stage (human),cartilage tissue_life cycle,triceps brachii_25-44 year-old human stage (human),right frontal lobe_human early adulthood stage (human),parotid gland_65-79 year-old human stage (human),thoracic mammary gland_human early adulthood stage (human),blood_infant stage,vagina_human middle aged stage (human),...,right lobe of liver_25-44 year-old human stage (human),ventral tegmental area_25-44 year-old human stage (human),corpus callosum_young adult stage (human),neocortex_fifth LMP month human stage (human),cerebellar hemisphere_human late adulthood stage (human),nasal cavity epithelium_human adult stage (human),esophagogastric junction muscularis propria_human adult stage (human),epithelium of mammary gland_life cycle,right lobe of liver_human middle aged stage (human),nucleus accumbens_young adult stage (human)
HMGB3P4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
PIK3C2B,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ITLN1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
MIR490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ACAA1,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [27]:
binary_matrix.shape

(34668, 1077)

# Save Binary Matrix

In [28]:
filename = path+'bgee_human_sample_binary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
binary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene Set Library

In [29]:
name = 'bgee_human_sample_gene_set'

In [30]:
uf.createUpGeneSetLib(binary_matrix, path, name)

# Create Attribute Library

In [31]:
name = 'bgee_human_sample_attribute_set'

In [32]:
uf.createUpAttributeSetLib(binary_matrix, path, name)

# Create Gene List

In [33]:
gene_list = uf.createGeneList(binary_matrix)

In [34]:
gene_list.head()

,GeneSym,GeneID
0,HMGB3P4,1.00874e+08
1,PIK3C2B,5287
2,ITLN1,55600
3,MIR490,574443
4,ACAA1,30


In [35]:
gene_list.shape

(34668, 2)

# Save Gene List

In [36]:
filename = path+'bgee_human_sample_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Attribute List

In [37]:
attribute_list = uf.createAttributeList(binary_matrix, metaData=meta_data)

In [38]:
attribute_list.head()

,Anatomical entity name,Developmental stage name
Attributes,,
epithelium of bronchus_65-79 year-old human stage (human),epithelium of bronchus,65-79 year-old human stage (human)
small intestine Peyer's patch_human late adulthood stage (human),small intestine Peyer's patch,human late adulthood stage (human)
frontal cortex_6-12 year-old child stage (human),frontal cortex,6-12 year-old child stage (human)
cartilage tissue_life cycle,cartilage tissue,life cycle
triceps brachii_25-44 year-old human stage (human),triceps brachii,25-44 year-old human stage (human)


In [39]:
attribute_list.shape

(1077, 2)

# Save Attribute List

In [40]:
filename = path+'bgee_human_sample_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Attribute Similarity matrix

In [41]:
attribute_similarity_matix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard')

In [42]:
attribute_similarity_matix.head()

,epithelium of bronchus_65-79 year-old human stage (human),small intestine Peyer's patch_human late adulthood stage (human),frontal cortex_6-12 year-old child stage (human),cartilage tissue_life cycle,triceps brachii_25-44 year-old human stage (human),right frontal lobe_human early adulthood stage (human),parotid gland_65-79 year-old human stage (human),thoracic mammary gland_human early adulthood stage (human),blood_infant stage,vagina_human middle aged stage (human),...,right lobe of liver_25-44 year-old human stage (human),ventral tegmental area_25-44 year-old human stage (human),corpus callosum_young adult stage (human),neocortex_fifth LMP month human stage (human),cerebellar hemisphere_human late adulthood stage (human),nasal cavity epithelium_human adult stage (human),esophagogastric junction muscularis propria_human adult stage (human),epithelium of mammary gland_life cycle,right lobe of liver_human middle aged stage (human),nucleus accumbens_young adult stage (human)
,,,,,,,,,,,,,,,,,,,,,
epithelium of bronchus_65-79 year-old human stage (human),1.000000,0.341008,0.568186,0.074324,0.317654,0.417575,0.610665,0.365212,0.610566,0.351898,...,0.447574,0.580455,0.429831,0.574818,0.350696,0.874476,0.318729,0.625629,0.397453,0.484161
small intestine Peyer's patch_human late adulthood stage (human),0.341008,1.000000,0.411510,0.032146,0.116359,0.726708,0.366672,0.843940,0.427424,0.858408,...,0.697030,0.419248,0.644607,0.454189,0.810376,0.323004,0.869463,0.520758,0.781100,0.296581
frontal cortex_6-12 year-old child stage (human),0.568186,0.411510,1.000000,0.067426,0.255179,0.532970,0.667530,0.434804,0.660308,0.423951,...,0.511581,0.808254,0.556940,0.818935,0.439439,0.556545,0.385337,0.697684,0.461358,0.677025
cartilage tissue_life cycle,0.074324,0.032146,0.067426,1.000000,0.103014,0.040907,0.075440,0.034375,0.064222,0.033021,...,0.043765,0.068382,0.044076,0.064729,0.033760,0.076066,0.029875,0.059647,0.038005,0.078604
triceps brachii_25-44 year-old human stage (human),0.317654,0.116359,0.255179,0.103014,1.000000,0.147558,0.289987,0.124900,0.239142,0.119374,...,0.157786,0.257078,0.159016,0.232422,0.121983,0.326125,0.108388,0.213641,0.137816,0.308273


# Save Attribute Similarity Matrix

In [43]:
filename = path+'bgee_human_sample_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene Similarity Matrix

In [44]:
gene_similarity_matix = uf.createSimilarityMatrix(binary_matrix, 'jaccard')

In [45]:
gene_similarity_matix.head()

,HMGB3P4,PIK3C2B,ITLN1,MIR490,ACAA1,MIR5580,WDR88,STRAP,GGT5,FDPSP1,...,C1orf228,BPIFB3,CLK2P1,RPSAP10,ARL13A,CCT6P3,CSMD2,SACS-AS1,TRAJ30,UBE2FP1
,,,,,,,,,,,,,,,,,,,,,
HMGB3P4,1.000000,0.312500,0.524851,0.225914,0.296827,0.227119,0.580087,0.294715,0.426036,0.051724,...,0.547170,0.056478,0.114094,0.170648,0.756598,0.723192,0.402685,0.305970,0.508091,0.405063
PIK3C2B,0.312500,1.000000,0.507511,0.085129,0.945863,0.077586,0.474138,0.941117,0.700637,0.016164,...,0.566058,0.030172,0.045259,0.057112,0.332974,0.430571,0.578158,0.253233,0.189655,0.754555
ITLN1,0.524851,0.507511,1.000000,0.158333,0.486708,0.146138,0.695009,0.483249,0.614306,0.027140,...,0.736207,0.058700,0.088050,0.111111,0.562624,0.599271,0.547655,0.398821,0.340862,0.640387
MIR490,0.225914,0.085129,0.158333,1.000000,0.080860,0.208000,0.174208,0.080285,0.117211,0.010753,...,0.149057,0.114583,0.100000,0.211009,0.235669,0.197007,0.136364,0.221790,0.294416,0.111425
ACAA1,0.296827,0.945863,0.486708,0.080860,1.000000,0.073695,0.450358,0.984818,0.682977,0.015353,...,0.540900,0.028659,0.042989,0.054248,0.316274,0.410440,0.557260,0.240532,0.180143,0.725691


# Save Gene Similarity Matrix

In [46]:
filename = path+'bgee_human_sample_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [47]:
name = 'bgee_human_sample_gene_attribute_edge_list'

In [48]:
uf.createGeneAttributeEdgeList(binary_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  1077 Out of 1077   

 The number of statisticaly relevent gene-attribute associations is: 14983142
